****Project Brief****
**Problem**: 

Choice Overload. Users waste more time searching than reading because of decision paralysis.

**Solution**: 

A context-aware engine that replaces choice fatigue with easy, simple, situational matching (mood, time, environment e.g. bedtime, 10mins, long holiday/travelling).

**Goal**: 

Reduce decision fatigue and help readers find the perfect book for their current moment. Additionally making the purchase easier as the website takes the customers directly to Amazon Purchase link. 

****Goodreads Webscraping****
"Best Book Ever" list


- Genre 
- Title 
- Author
- Rating
- Rating counts 
- Description 
- Page numbers 
- ISBN
- Language 
- Published Year 
- Book Cover Image 
- Link to the book 

****Open Library API***
- Fiction / 600 
- Biography / 400
- History /100
  
Book data acquired 

- title
- author
- genre
- rating_average
- rating_counts
- description
- page_numbers
- isbn
- language
- published_year
- cover_url
- book_link

In [24]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [6]:
#Checking if the webscraping works 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
r = requests.get("https://www.goodreads.com/", headers=headers)
print (r.status_code)

200


soup = BeautifulSoup(r.text, 'html.parser' ) 
print (soup.prettify())

In [ ]:
#create the dictionary of genre list 
genres_list = {}
for a in soup.select("div a.gr-hyperlink href=genres/art"):

In [21]:
genre_art = []

for a in soup.select('a.gr-hyperlink[href="/genres/art"]'):
    text = a.get_text(strip=True)
    
    if text:
        genre_art.append(text)

print(genre_art)

['Art']


#genre_art = []
#----------------------------
#for a in soup.select('a.gr-hyperlink[href="/genres/art"]'):
#text = a.get_text(strip=True)
#if text:
   #     genre_art.append(text)

#print(genre_art)

In [22]:
#Best Book Ever List from GoodReads 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
r = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever", headers=headers)
print (r.status_code)

200


In [ ]:
#Prettyfing the Best Book Ever Page 
soup = BeautifulSoup(r.text, 'html.parser' ) 
#print (soup.prettify())

In [26]:
#Best Book Ever List from GoodReads 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
r = requests.get("https://www.goodreads.com/book/show/2767052-the-hunger-games", headers=headers)
print (r.status_code)

200


In [ ]:
#Prettifying Hanger Games  
soup = BeautifulSoup(r.text, 'html.parser' ) 
#print (soup.prettify())

In [ ]:
#Testing to see if we can scrape the title only, it did not return result 
results_list = [] 
for a in soup.select('h1.Text_title1'):
    text = a.get_text(strip=True)

    results_list.append(text)

print(results_list)

In [29]:
#Web Scraping of Best Book Ever List 
# Base URL for the list
the_hunger_game = {}

base_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever"
page_to_scrape = 1  # Starting page

def scrape_book_details(book_url):
    # This simulates "clicking" into the book
    full_url = "https://www.goodreads.com" + book_url
    response = requests.get(full_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #Title 
    title = soup.find('h1', {'class': 'Text Text_title1'}) #this was the wrong position 
    return title.text.strip() if title else "No title"

# Pagination Loop
while page_to_scrape <= 2:  # Let's just do 2 pages for this example
    print(f"--- Scraping Page {page_to_scrape} ---")
    params = {'page': page_to_scrape}
    response = requests.get(base_url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 1. Find all book links on the list page
    book_links = soup.find_all('a', class_='bookTitle')

    for link in book_links:
        relative_url = link['href']
        title = link.find('span').text
        print(f"Clicking into: {title}")
        
        # 2. Go inside the book page
        desc = scrape_book_details(relative_url)
        print(f"Description found: {desc[:50]}...")
        
        # Respectful delay so you don't get banned
        all_books.append(the_hunger_game)
        time.sleep(1)

    page_to_scrape += 1

the_hunger_game

--- Scraping Page 1 ---
--- Scraping Page 2 ---


{}

In [7]:
#1000 Fiction Books from Open Library API 
import requests
import pandas as pd
import time

open_lib = []
target_count = 600
page = 1

#putting all the column names, so to avoid empty cells (only return the data when its not empty) 
fields = "title,author_name,subject,ratings_average,ratings_count,first_sentence,number_of_pages_median,isbn,language,first_publish_year,cover_i,key"

print("starting Data Collection...")

while len(open_lib) < target_count:
    url = f"https://openlibrary.org/search.json?subject=fiction&fields={fields}&page={page}&limit=1000" #genre is fiction here
    
    headers = {'User-Agent': 'bookrecommendation/1.0 (example@email.com)'} #this gives an user agent to make the access clear its human 
    
    try:
        response = requests.get(url, headers=headers, timeout=10) #if there is not response in 10seconds, show connection error 
    except requests.exceptions.RequestException as e: 
        print(f"❌ Connection error: {e}")
        break

    if response.status_code == 200: #if status code is 200, proceed 
        json_response = response.json()
        data = json_response.get('docs', []) #'docs' is where the book list is : "Each document specified listed in "docs"
        
        if not data:
            print("No more data available.")
            break

        for item in data:
            if len(open_lib) >= target_count:
                break

            # 2. Extracting data with safe defaults
            # Note: first_sentence is often a list, so we handle that specifically
            desc = item.get('first_sentence')
            description = desc[0] if isinstance(desc, list) else "No description available"

            row = {
                'Title': item.get('title'),
                'Author': item.get('author_name', ['N/A'])[0], #"Get the list of authors. If there are none, use ['N/A']. Then, just take the 1st one ([0])."
                'Genre': ", ".join(item.get('subject', [])[:3]), #Subjects are also lists. This takes the first 3 items ([:3]) unified with a comma.
                'Rating_Average': item.get('ratings_average'),
                'Rating_Counts': item.get('ratings_count'),
                'Description': description,
                'Page_Numbers': item.get('number_of_pages_median'), #OP has median of all versions e.g. hard/paper cover 
                'ISBN': item.get('isbn', ['N/A'])[0],
                'Language': item.get('language', ['N/A'])[0],
                'Published_Year': item.get('first_publish_year'),
                'Cover_URL': f"https://covers.openlibrary.org/b/id/{item.get('cover_i')}-L.jpg" if item.get('cover_i') else None,
                'Book_Link': f"https://openlibrary.org{item.get('key')}"
            }
        
            open_lib.append(row) #appending retrieved data to the list 
        
        print(f"Page {page} processed. Total items collected: {len(open_lib)}") #number of page collected 
        page += 1 #increment the page one by one 
        time.sleep(1) #sleep for 1 sec per page loading 
        
    elif response.status_code == 429: #if there is some distruption, wait for 20secs 
        print("Waiting 20 seconds...")
        time.sleep(20)
    else:
        print(f"❌ Error {response.status_code}. Stopping.")
        break

#converting to dataframe
op_fic_df = pd.DataFrame(open_lib)

op_fic_df.dropna(subset=['Title'], inplace=True) #if title does not exist, drop 

print("Collection Complete!")

op_fic_df.to_csv('Open_Library_Fiction_600.csv', index=False)

starting Data Collection...
Page 1 processed. Total items collected: 600
Collection Complete!


In [8]:
op_fic_df.head(10)

,Title,Author,Genre,Rating_Average,Rating_Counts,Description,Page_Numbers,ISBN,Language,Published_Year,Cover_URL,Book_Link
0,The Iron Heel,Jack London,"Revolutions, fiction, Oligarchy, fiction, Utop...",3.500000,20.0,"THE SOFT summer wind stirs the redwood, and Wi...",287,1548921947,eng,1907,https://covers.openlibrary.org/b/id/8243314-L.jpg,https://openlibrary.org/works/OL74502W
1,Игрокъ,Фёдор Михайлович Достоевский,"Translations into English, Continental europea...",3.714286,7.0,At length I returned from two weeks leave of a...,191,9798630153272,eng,1900,https://covers.openlibrary.org/b/id/3293339-L.jpg,https://openlibrary.org/works/OL166923W
2,Brood of the Witch-Queen,Sax Rohmer,"Fantasy, Fiction, Horror",4.600000,5.0,No description available,206,1985805111,eng,1924,https://covers.openlibrary.org/b/id/2011286-L.jpg,https://openlibrary.org/works/OL2288676W
3,The Napoleon of Notting Hill,Gilbert Keith Chesterton,"Fiction, Classic Literature, Science Fiction",4.571429,7.0,No description available,160,1445508265,eng,1904,https://covers.openlibrary.org/b/id/6980094-L.jpg,https://openlibrary.org/works/OL76473W
4,Emily of New Moon,Lucy Maud Montgomery,"Juvenile fiction, Child authors, Orphans",3.666667,9.0,"The house in the hollow was ""a mile from anywh...",339,1548912425,eng,1923,https://covers.openlibrary.org/b/id/14638065-L...,https://openlibrary.org/works/OL77781W
5,The Vampyre,John William Polidori,"Fiction, Incest, Vampires",3.583333,12.0,No description available,55,1517322030,fre,1819,https://covers.openlibrary.org/b/id/4871002-L.jpg,https://openlibrary.org/works/OL3625242W
6,The Sea Fairies,L. Frank Baum,"Children's stories, Juvenile fiction, Classic ...",3.333333,3.0,"""Nobody,"" said Cap'n Bill solemnly, ""ever sawr...",130,9781098604226,rus,1911,https://covers.openlibrary.org/b/id/1814237-L.jpg,https://openlibrary.org/works/OL262384W
7,Lilith,George MacDonald,"Fiction, romance, fantasy, Fiction, general, N...",3.200000,5.0,"I had just finished my studies at Oxford, and ...",270,1406530042,fre,1895,https://covers.openlibrary.org/b/id/14364546-L...,https://openlibrary.org/works/OL15437W
8,The Enchanted Castle,Edith Nesbit,"Fiction, Fantasy, Magic",4.181818,11.0,"There were three of them-Jerry, Jimmy, and Kat...",186,9781548958534,spa,1907,https://covers.openlibrary.org/b/id/6644514-L.jpg,https://openlibrary.org/works/OL99541W
9,Pollyanna,Eleanor Hodgman Porter,"Aunts, Cheerfulness, Classic Literature",3.809524,21.0,Miss Polly Harrington entered her kitchen a li...,194,1532762569,eng,1912,https://covers.openlibrary.org/b/id/902113-L.jpg,https://openlibrary.org/works/OL2775807W


In [ ]:
#History - Data Retriaval from Open Library API 
op_nonfic = []
target_count = 100
page = 1

#putting all the column names, so to avoid empty cells (only return the data when its not empty) 
fields = "title,author_name,subject,ratings_average,ratings_count,first_sentence,number_of_pages_median,isbn,language,first_publish_year,cover_i,key"

print("starting Data Collection...")

while len(op_nonfic) < target_count:
    url = f"https://openlibrary.org/search.json?subject=history&fields={fields}&page={page}&limit=100" #genre is fiction here
    
    headers = {'User-Agent': 'bookrecommendation/1.0 (example@email.com)'} #this gives an user agent to make the access clear its human 
    
    try:
        response = requests.get(url, headers=headers, timeout=10) #if there is not response in 10seconds, show connection error 
    except requests.exceptions.RequestException as e: 
        print(f"❌ Connection error: {e}")
        break

    if response.status_code == 200: #if status code is 200, proceed 
        json_response = response.json()
        data = json_response.get('docs', []) #'docs' is where the book list is : "Each document specified listed in "docs"
        
        if not data:
            print("No more data available.")
            break

        for item in data:
            if len(op_nonfic) >= target_count:
                break

            # 2. Extracting data with safe defaults
            # Note: first_sentence is often a list, so we handle that specifically
            desc = item.get('first_sentence')
            description = desc[0] if isinstance(desc, list) else "No description available"

            row = {
                'Title': item.get('title'),
                'Author': item.get('author_name', ['N/A'])[0], #"Get the list of authors. If there are none, use ['N/A']. Then, just take the 1st one ([0])."
                'Genre': ", ".join(item.get('subject', [])[:3]), #Subjects are also lists. This takes the first 3 items ([:3]) unified with a comma.
                'Rating_Average': item.get('ratings_average'),
                'Rating_Counts': item.get('ratings_count'),
                'Description': description,
                'Page_Numbers': item.get('number_of_pages_median'), #OP has median of all versions e.g. hard/paper cover 
                'ISBN': item.get('isbn', ['N/A'])[0],
                'Language': item.get('language', ['N/A'])[0],
                'Published_Year': item.get('first_publish_year'),
                'Cover_URL': f"https://covers.openlibrary.org/b/id/{item.get('cover_i')}-L.jpg" if item.get('cover_i') else None,
                'Book_Link': f"https://openlibrary.org{item.get('key')}"
            }
        
            op_nonfic.append(row) #appending retrieved data to the list 
        
        print(f"Page {page} processed. Total items collected: {len(op_nonfic)}") #number of page collected 
        page += 1 #increment the page one by one 
        time.sleep(1) #sleep for 1 sec per page loading 
        
    elif response.status_code == 429: #if there is some distruption, wait for 20secs 
        print("Waiting 20 seconds...")
        time.sleep(20)
    else:
        print(f"❌ Error {response.status_code}. Stopping.")
        break

#converting to dataframe
op_hist_df = pd.DataFrame(op_nonfic)

op_hist_df.dropna(subset=['Title'], inplace=True) #if title does not exist, drop 

print("Collection Complete!")

op_hist_df.to_csv('Open_Library_History_600.csv', index=False)

In [12]:
op_hist_df

,Title,Author,Genre,Rating_Average,Rating_Counts,Description,Page_Numbers,ISBN,Language,Published_Year,Cover_URL,Book_Link
0,The Story of Philosophy,Will Durant,"Philosophers, Philosophy, History",4.200000,15.0,IF YOU look at a map of Europe you will observ...,543,1512110353,urd,1926,https://covers.openlibrary.org/b/id/405360-L.jpg,https://openlibrary.org/works/OL1073898W
1,Записки изъ подполья,Фёдор Михайлович Достоевский,"Fiction, History, Officials and employees",4.250000,4.0,The ellipsis after the opening sentence of Not...,130,1503055582,eng,1864,https://covers.openlibrary.org/b/id/12644033-L...,https://openlibrary.org/works/OL25571500W
2,The Enduring Vision,Paul S. Boyer,"Textbooks, History, United states, history",4.625000,8.0,No description available,610,9780618218615,eng,1987,https://covers.openlibrary.org/b/id/8237922-L.jpg,https://openlibrary.org/works/OL2024851W
3,La conquête du pain,Peter Kropotkin,"History, Communism, Anarchism",4.071429,14.0,به شیوا\r\nو نارنج هزاردانهٔ آفتاب\r\n\r\nر. الف,234,8897011268,ita,1892,https://covers.openlibrary.org/b/id/7296134-L.jpg,https://openlibrary.org/works/OL1105382W
4,A Study of History,Arnold J. Toynbee,"Civilization, Philosophy, History",4.500000,8.0,HISTORIANS generally illustrate rather than co...,576,9780192152190,eng,1900,https://covers.openlibrary.org/b/id/121255-L.jpg,https://openlibrary.org/works/OL1108523W
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Paradise Lost,John Milton,"Bible, Biography, Criticism and interpretation",3.828571,35.0,"This first book proposes, first in brief, the ...",333,0198319126,mal,1667,https://covers.openlibrary.org/b/id/5992814-L.jpg,https://openlibrary.org/works/OL810991W
96,Five Children and It,Edith Nesbit,"Fiction, Fairies, Wishes",4.000000,7.0,"The house was three miles from the station, bu...",150,1306356903,eng,1905,https://covers.openlibrary.org/b/id/28174-L.jpg,https://openlibrary.org/works/OL99499W
97,Othello,William Shakespeare,"Drama, Othello (Fictitious character), Printing",3.780488,41.0,"'Othello', in the words of Edward Pechter, 'ha...",180,9788495882226,jpn,1622,https://covers.openlibrary.org/b/id/7165018-L.jpg,https://openlibrary.org/works/OL258850W
98,Sonnets,William Shakespeare,"English Sonnets, History and criticism, Biblio...",4.166666,12.0,"From the fairest creatures we desire increase,",168,9789504359470,ukr,1609,https://covers.openlibrary.org/b/id/8222090-L.jpg,https://openlibrary.org/works/OL362706W


In [13]:
#biography 
op_bio = []
target_count = 400
page = 1

#putting all the column names, so to avoid empty cells (only return the data when its not empty) 
fields = "title,author_name,subject,ratings_average,ratings_count,first_sentence,number_of_pages_median,isbn,language,first_publish_year,cover_i,key"

print("starting Data Collection...")

while len(op_bio) < target_count:
    url = f"https://openlibrary.org/search.json?subject=biography&fields={fields}&page={page}&limit=400" #genre is fiction here
    
    headers = {'User-Agent': 'bookrecommendation/1.0 (example@email.com)'} #this gives an user agent to make the access clear its human 
    
    try:
        response = requests.get(url, headers=headers, timeout=10) #if there is not response in 10seconds, show connection error 
    except requests.exceptions.RequestException as e: 
        print(f"❌ Connection error: {e}")
        break

    if response.status_code == 200: #if status code is 200, proceed 
        json_response = response.json()
        data = json_response.get('docs', []) #'docs' is where the book list is : "Each document specified listed in "docs"
        
        if not data:
            print("No more data available.")
            break

        for item in data:
            if len(op_bio) >= target_count:
                break

            desc = item.get('first_sentence')
            description = desc[0] if isinstance(desc, list) else "No description available"

            row = {
                'Title': item.get('title'),
                'Author': item.get('author_name', ['N/A'])[0], #"Get the list of authors. If there are none, use ['N/A']. Then, just take the 1st one ([0])."
                'Genre': ", ".join(item.get('subject', [])[:3]), #Subjects are also lists. This takes the first 3 items ([:3]) unified with a comma.
                'Rating_Average': item.get('ratings_average'),
                'Rating_Counts': item.get('ratings_count'),
                'Description': description,
                'Page_Numbers': item.get('number_of_pages_median'), #OP has median of all versions e.g. hard/paper cover 
                'isbn': item.get('isbn', ['N/A'])[0],
                'Language': item.get('language', ['N/A'])[0],
                'Published_Year': item.get('first_publish_year'),
                'Cover_URL': f"https://covers.openlibrary.org/b/id/{item.get('cover_i')}-L.jpg" if item.get('cover_i') else None,
                'Book_Link': f"https://openlibrary.org{item.get('key')}"
            }
        
            op_bio.append(row) #appending retrieved data to the list 
        
        print(f"Page {page} processed. Total items collected: {len(op_bio)}") #number of page collected 
        page += 1 #increment the page one by one 
        time.sleep(1) #sleep for 1 sec per page loading 
        
    elif response.status_code == 429: #if there is some distruption, wait for 20secs 
        print("Waiting 20 seconds...")
        time.sleep(20)
    else:
        print(f"❌ Error {response.status_code}. Stopping.")
        break

#converting to dataframe
op_bio_df = pd.DataFrame(op_bio)

op_bio_df.dropna(subset=['Title'], inplace=True) #if title does not exist, drop 

print("Collection Complete!")

op_bio_df.to_csv('Open_Library_Biography_400.csv', index=False)

starting Data Collection...
Page 1 processed. Total items collected: 400
Collection Complete!


In [14]:
op_bio_df

,Title,Author,Genre,Rating_Average,Rating_Counts,Description,Page_Numbers,ISBN,Language,Published_Year,Cover_URL,Book_Link
0,"The life of Olaudah Equiano, or Gustavus Vassa...",Olaudah Equiano,"Slavery, Voyages and travels, Social life and ...",4.000000,5.0,PERMIT me with the greatest deference and resp...,241,1528705653,eng,1789,https://covers.openlibrary.org/b/id/2074044-L.jpg,https://openlibrary.org/works/OL743333W
1,Autobiography of a Yogi,Yogananda Paramahansa,"Autobiography, Yogis, Yoga",4.186047,43.0,THE CHARACTERISTIC FEATURES of Indian culture ...,514,9781565897342,eng,1946,https://covers.openlibrary.org/b/id/805448-L.jpg,https://openlibrary.org/works/OL528094W
2,Lives,Plutarch,"Biography, Classical biography, Early works to...",3.375000,8.0,"As geographers, Sosius, crowd into the edges o...",483,9780375756764,grc,1564,https://covers.openlibrary.org/b/id/10679669-L...,https://openlibrary.org/works/OL2521179W
3,Narrative of the life of Frederick Douglass,Frederick Douglass,"Douglass, frederick, 1818-1895, Biography, Afr...",4.333334,12.0,Frederick Douglass was the most important Afri...,127,9798698409281,und,1845,https://covers.openlibrary.org/b/id/8247724-L.jpg,https://openlibrary.org/works/OL69178W
4,Les confessions,Jean-Jacques Rousseau,"French Authors, Biography, Correspondence",4.500000,4.0,No description available,606,034180844X,eng,1782,https://covers.openlibrary.org/b/id/6371045-L.jpg,https://openlibrary.org/works/OL80592W
...,...,...,...,...,...,...,...,...,...,...,...,...
395,Zlatin dnevnik,Zlata Filipović,"Children and war, Diaries, Bosnian Personal na...",5.000000,1.0,"Behind me-a long, hot summer and the happy day...",200,9578829086,chi,1993,https://covers.openlibrary.org/b/id/3899848-L.jpg,https://openlibrary.org/works/OL3029795W
396,The Journals of Sylvia Plath,Ted Hughes,"American Poets, Biography, Diaries",NaN,NaN,No description available,370,3492227783,eng,1982,https://covers.openlibrary.org/b/id/8222248-L.jpg,https://openlibrary.org/works/OL1865540W
397,Lady sings the blues,Billie Holiday,"Jazz, Zangeressen, Correspondence, reminiscences",4.333334,3.0,No description available,203,0380004917,eng,1956,https://covers.openlibrary.org/b/id/93864-L.jpg,https://openlibrary.org/works/OL5603902W
398,Small sacrifices,Ann Rule,"Murder, Trials (Murder), Case studies",4.400000,10.0,gency room doctor; his assessment of patient's...,494,0751535567,eng,1987,https://covers.openlibrary.org/b/id/291624-L.jpg,https://openlibrary.org/works/OL892130W


In [ ]:
gr_df = pd.read_csv ('../data/processed/goodreads_books.csv')
gr_df.info()

In [22]:
#Standardising Columns to lower cases 
op_bio_df.columns = op_bio_df.columns.str.lower()
gr_df.columns = gr_df.columns.str.lower()
op_hist_df.columns = op_hist_df.columns.str.lower()
op_fic_df.columns = op_fic_df.columns.str.lower()

In [31]:
gr_df['isbn'] = gr_df['isbn'].astype('object')
op_bio_df['isbn'] = op_bio_df['isbn'].astype('object')
op_hist_df['isbn'] = op_hist_df['isbn'].astype('object')
op_fic_df['isbn'] = op_fic_df['isbn'].astype('object')

In [37]:
api_df = pd.concat([op_bio_df, op_hist_df, op_fic_df], ignore_index=True)
api_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           1100 non-null   object 
 1   author          1100 non-null   object 
 2   genre           1100 non-null   object 
 3   rating_average  1073 non-null   float64
 4   rating_counts   1073 non-null   float64
 5   description     1100 non-null   object 
 6   page_numbers    1100 non-null   int64  
 7   isbn            1100 non-null   object 
 8   language        1100 non-null   object 
 9   published_year  1100 non-null   int64  
 10  cover_url       1099 non-null   object 
 11  book_link       1100 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 103.3+ KB


In [40]:
api_df.head(10)

,title,author,genre,rating_average,rating_counts,description,page_numbers,isbn,language,published_year,cover_url,book_link
0,"The life of Olaudah Equiano, or Gustavus Vassa...",Olaudah Equiano,"Slavery, Voyages and travels, Social life and ...",4.000000,5.0,PERMIT me with the greatest deference and resp...,241,1528705653,eng,1789,https://covers.openlibrary.org/b/id/2074044-L.jpg,https://openlibrary.org/works/OL743333W
1,Autobiography of a Yogi,Yogananda Paramahansa,"Autobiography, Yogis, Yoga",4.186047,43.0,THE CHARACTERISTIC FEATURES of Indian culture ...,514,9781565897342,eng,1946,https://covers.openlibrary.org/b/id/805448-L.jpg,https://openlibrary.org/works/OL528094W
2,Lives,Plutarch,"Biography, Classical biography, Early works to...",3.375000,8.0,"As geographers, Sosius, crowd into the edges o...",483,9780375756764,grc,1564,https://covers.openlibrary.org/b/id/10679669-L...,https://openlibrary.org/works/OL2521179W
3,Narrative of the life of Frederick Douglass,Frederick Douglass,"Douglass, frederick, 1818-1895, Biography, Afr...",4.333334,12.0,Frederick Douglass was the most important Afri...,127,9798698409281,und,1845,https://covers.openlibrary.org/b/id/8247724-L.jpg,https://openlibrary.org/works/OL69178W
4,Les confessions,Jean-Jacques Rousseau,"French Authors, Biography, Correspondence",4.500000,4.0,No description available,606,034180844X,eng,1782,https://covers.openlibrary.org/b/id/6371045-L.jpg,https://openlibrary.org/works/OL80592W
5,The Story of Philosophy,Will Durant,"Philosophers, Philosophy, History",4.200000,15.0,IF YOU look at a map of Europe you will observ...,543,1512110353,urd,1926,https://covers.openlibrary.org/b/id/405360-L.jpg,https://openlibrary.org/works/OL1073898W
6,Terre des hommes,Antoine de Saint-Exupéry,"20th century, Aeronautics, Air pilots",4.235294,17.0,In 1926 I was enrolled as student airline pilo...,222,9781388227470,fre,1936,https://covers.openlibrary.org/b/id/967072-L.jpg,https://openlibrary.org/works/OL86701W
7,Der Antichrist,Friedrich Nietzsche,"Christianity, Controversial literature, Antich...",3.750000,20.0,LET us look each other in the face.,92,1727031393,eng,1895,https://covers.openlibrary.org/b/id/1759313-L.jpg,https://openlibrary.org/works/OL15692719W
8,American notes,Charles Dickens,"Description and travel, Social life and custom...",NaN,NaN,I SHALL never forget the one-fourth serious an...,266,9781727717983,eng,1800,https://covers.openlibrary.org/b/id/8236357-L.jpg,https://openlibrary.org/works/OL8300173W
9,Incidents in the Life of a Slave Girl,Harriet A. Jacobs,"Women slaves, United States, Slaves",3.888889,9.0,I was born a slave; but I never knew it till s...,236,1547127252,dut,1861,https://covers.openlibrary.org/b/id/411542-L.jpg,https://openlibrary.org/works/OL28623W
